# Importing

In [13]:
import numpy as np
import matplotlib.pyplot as mpl

# Definfing Variables

In [14]:
StartingTempMol = 283.15 #change this to Temp we are using
AirTemp= 277.55 # Air temperature in Kelvin
EndDistance= 135 # End distance in meters
TankRadi= 13.5 # Tank radius in meters
TankHeight= 15 # Tank height in meters
StartingVelMol= 100 #change this to velocity we are using
density = 1415 # Density in kg/m^3

# Initial conditions

In [ ]:
# Spatial variables
N=100000 # Number of spatial steps
a=EndDistance/(N-1) # Size of a spatial node
indexToPos = np.arange(0,135+a,a) #Converts a 0-based index to a position in meters
def posToIndex(i): #Converts a position in meters to a 0-based index
    return np.floor(i/a)

# Temporal variables

h=0.01 # Time step in seconds

# Tracking position
hasMolasses = np.zeros(N,dtype=bool) # contains true if there is molasses at a given 
                                     # point, false otherwise
hasMolasses[0] = True # Initially, there is only molasses at the hole

# Heat equation
temp = np.full(N, AirTemp, dtype=np.float64) # Stores the temperature at the given index
temp[0] = StartingTempMol # At the hole, the molasses is the same temperature as in the tank

# Viscosity equation
visc = np.zeros(N, dtype=np.float64)

# Velocity equation
vel = np.zeros(N, dtype=np.float64)
vel[0] = StartingVelMol

# Box of Functions

In [ ]:
MAX_IND = 0 # Stores the maximum index that is filled with molasses. Will only be valid
            # in 1D.

def xStep(): # Updates the molasses progress bar and the value MAX_IND which all other quantities depend on
    
    v = vel[MAX_IND]
    dx = v*h
    dind = posToIndex(dx)
    for i in range(MAX_IND+1, MAX_IND + 1 + dind):
        hasMolasses[i] = True
    
    return dind, hasMolasses

#----------------- Heat Equation Below-----------------# 

heatcon = h/(a*a)

def alpha(T):
    k = 1.3e-3*T + 0.403
    return k/(2.56e3*1415)

def HeatStep():
    alphaarr = alpha(temp)
    Tp = np.zeros(N, dtype=np.float64)
    Tp[0] = StartingTempMol
    Tp[1:MAX_IND] = temp[1:MAX_IND] + heatcon*alphaarr*(temp[2:MAX_IND+1] + temp[:MAX_IND-1] - 2*temp[1:MAX_IND])
    temp = Tp

#-----------------Viscosity Equation Below------------#

E0 = 53.074
R = 8.314
mu0 = 2.3e-9

def ViscEQ(T):
    return mu0 * np.exp(-E0/(R*T))

#----------------Equation of Motion Below-------------#

# Note: Confined to 1 dimension, velocity is a scalar quantity, and gravity isn't a consideration



def velStep(): # calculates the velocity at any position within any one timestep 
    
    muarr = ViscEQ(temp)
    velp = np.zeros(N, dtype=np.float64)
    velp[0] = StartingVelMol
    velp[1:MAX_IND] = vel[1:MAX_IND] + (-muarr*c)*(vel[2:MAX_IND+1] + vel[:MAX_IND-1] - 2*vel[1:MAX_IND])/density
    vel = velp



This next code cell should run all of the above functions in a while loop over N. This accounts for all spatial steps. I haven't yet figured out the specific architecture for this routine but if MAX_IND is updating properly, it should be easy enough. Those three lines at the bottom need to be there in order to update dind, MAX_IND, and hasMolasses, and need to occur in that specific order.

In [ ]:

while MAX_IND < N: 





dind = xStep()[0]
hasMolasses = xStep()[1]

MAX_IND += dind

This next code cell is the experimental 2D stuff I wanted to do. I haven't given up on it yet, but in the wake of revising the structure of the code I will need to re-evaluate how this function is meant to work.

In [ ]:
# The stuff below is experimental
# Trying to define a function that produces a 2D array of velocity values
# With each row representing velocity values at a particular timestep
# and each column representing a specific value in the simulation
# (i.e. all column 1 entries are at x meters, column 2 is at x + a, etc.)
# If this works we can do it for viscosity and temperature as well


def velMap(duration):

    dt = h

    t = 0

    velBox = [vel] # array of velocity arrays: each row is one timestep

    while t < duration:
        
        xStep()

        if MAX_IND > 2:
            newvel = velStep()
            velBox += [vel]
        
        t += dt
    
    return velBox
mpl.imshow(velMap(1),aspect="auto")
mpl.show()
    
